In [133]:
import os
import numpy as np
import pandas as pd
import tempfile

import pandapower as pp
import pandapower.networks as nw
from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl
import pandapower.plotting as pplt
import matplotlib.pyplot as plt
from pandapower.plotting.plotly import pf_res_plotly

#creating our net
net = nw.create_cigre_network_mv()

#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')

#print(net)

#ax = pplt.simple_plot(net, show_plot = False)
#clc = pplt.create_line_collection(net, color ="g", linewidth = 3.,use_bus_geodata=(True))
#pplt.draw_collections([clc], ax=ax)
#plt.show()

#Adding CS at rated power (6.6 kW)


max_line_loading=100

max_vm_pu= 1.1

min_vm_pu= 0.9



In [134]:

#importing the active and reactive power of loads from excel
df_active = pd.read_csv('active_power_time_series_ev_load.csv')
df_reactive = pd.read_csv('reactive_power_time_series_ev_load.csv')
# create the data source from
ds_active = DFData(df_active)
ds_reactive = DFData(df_reactive)

def timeseries_example(output_dir):
    
    #defing the time step
    n_timesteps = 24
    
    # 3. create controllers (to control P values of the load) ds_p is for active power and ds_q for reactive power
    create_controllers(net, ds_active, ds_reactive)

    # time steps to be calculated. Could also be a list with non-consecutive time steps
    time_steps = range(0, n_timesteps)

    # 4. the output writer with the desired results to be stored to files.
    ow = create_output_writer(net, time_steps, output_dir=output_dir)

    # 5. the main time series function
    run_timeseries(net, time_steps) 
    
    
load_index=net.load.index
load_name=net.load.name
def create_controllers(net,ds_active,ds_reactive):
    for i in range(len(load_index)):
        ConstControl(net, element='load', variable='p_mw', element_index=load_index[i],
                 data_source=ds_active, profile_name=load_name[i])
        ConstControl(net, element='load', variable='q_mvar', element_index=load_index[i],
                 data_source=ds_reactive, profile_name=load_name[i])

def create_output_writer(net, time_steps, output_dir):
    ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())
    # these variables are saved to the harddisk after / during the time series loop
    ow.log_variable('res_load', 'p_mw')
    ow.log_variable('res_bus', 'vm_pu')
    ow.log_variable('res_line', 'loading_percent')
    ow.log_variable('res_line', 'i_ka')
    ow.log_variable('res_trafo','loading_percent')
    return ow


#

In [135]:
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

Results can be found in your local temp folder: time_series_ev



100%|██████████████████████████████████████████| 24/24 [00:00<00:00, 118.32it/s]


In [136]:
ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)
    
vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

if any(line_loading.max()>100):
        print("Overloaded lines")

elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")

In [137]:
#lowlimit=350
#uplimit=400

#Bus CS 14 (0.63 MVA) - trafo id: 2

#for i in range (1,41):
 #   l = str(i)
  #  pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 11 (0.4 MVA) - trafo id: 3
#for i in range (uplimit,uplimit+20):
#    l = str(i)
 #   pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

#Bus CS 5 (0.4 MVA) - trafo id: 4
#for i in range (uplimit+20,uplimit+40):
  #  l = str(i)
  #  pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
#Bus CS 4 (0.25 MVA) - trafo id: 5
#for i in range (uplimit+40,uplimit+50):
 #   l = str(i)
 #   pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
#Bus CS 9 (0.25 MVA)- trafo id: 6
#for i in range (uplimit+50,uplimit+60):
  #  l = str(i)
 #   pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)




In [138]:

lowlimit1=368
uplimit1=400

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (1,lowlimit1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit1
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit1:
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit1:
        print("no maximum has been found")
        
#creating our net
net = nw.create_cigre_network_mv()


#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

369
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 84.78it/s]


370
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 80.26it/s]


371
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 63.94it/s]


voltage below minimum
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 97.32it/s]


In [139]:
c15=c

lowlimit2=c15+205
uplimit2=c15+250

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (c15,lowlimit2):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit2
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit2:
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit2:
        print("no maximum has been found")
        
c16=c-2
EVs=c16-c15        
print(EVs)        
        
#creating our net
net = nw.create_cigre_network_mv()


#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

577
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 78.45it/s]


578
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 69.78it/s]


voltage below minimum
205
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 71.00it/s]


In [140]:
vm_pu.min()

0     1.030000
1     1.019881
2     1.007501
3     0.987814
4     0.987359
5     0.987048
6     0.986689
7     0.983246
8     0.983287
9     0.982257
10    0.980068
11    0.979217
12    1.018629
13    0.997813
14    0.985152
15    0.900496
16    0.900919
17    0.987025
18    0.987336
19    0.982234
dtype: float64

In [141]:


lowlimit3=c16+10
uplimit3=c16+20


for i in range (c16,lowlimit3):
    l = str(i)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit3
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit3:
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit3:
        print("no maximum has been found")
        
c17=c-1
EVs=c-c16        
print(EVs)        
        
#creating our net
net = nw.create_cigre_network_mv()


#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c16,c17):
    l = str(i)
    pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

587
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 76.33it/s]


voltage below minimum
11
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 80.71it/s]


In [142]:
vm_pu.min()

0     1.030000
1     1.019832
2     1.007180
3     0.987082
4     0.986574
5     0.986215
6     0.985855
7     0.982508
8     0.982549
9     0.981517
10    0.979326
11    0.978474
12    1.018629
13    0.997813
14    0.985152
15    0.900496
16    0.900058
17    0.983753
18    0.986551
19    0.981495
dtype: float64

In [143]:


lowlimit4=c17
uplimit4=c17+30

#Bus CS 14 (0.63 MVA) - trafo id: 2

#for i in range (c17,lowlimit4):
#    l = str(i)
#    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit4
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit4:
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit4:
        print("no maximum has been found")
        
c18=c
EVs=c-c17        
print(EVs)        
        
#creating our net
net = nw.create_cigre_network_mv()


#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c16,c17):
   l = str(i)
   pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c17,c18):
    l = str(i)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

587
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 73.76it/s]


voltage below minimum
1
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 81.46it/s]


In [144]:
vm_pu.min()

0     1.030000
1     1.019827
2     1.007148
3     0.987010
4     0.986497
5     0.986138
6     0.985778
7     0.982436
8     0.982476
9     0.981445
10    0.979253
11    0.978401
12    1.018629
13    0.997813
14    0.985152
15    0.900496
16    0.899973
17    0.983675
18    0.986087
19    0.981422
dtype: float64

In [145]:


lowlimit5=c18+5
uplimit5=c17+30

#Bus CS 14 (0.63 MVA) - trafo id: 2

for i in range (c18,lowlimit5):
    l = str(i)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

c=lowlimit5
while all(vm_pu.max()<max_vm_pu) and all(line_loading.max()<100) and all(vm_pu.min()>min_vm_pu) and c<uplimit5:
    
    c=c+1
    print(c)
    k=str(c)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+k)
    
    output_dir = os.path.join("time_series_ev")
    print("Results can be found in your local temp folder: {}".format(output_dir))
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    timeseries_example(output_dir)

    ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
    line_loading = pd.read_excel(ll_file, index_col=0)
    
    vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
    vm_pu = pd.read_excel(vm_pu_file, index_col=0)

    if any(line_loading.max()>100):
        print("Overloaded lines")

    elif any(vm_pu.max()>max_vm_pu):
        print("voltage over maximum")

    elif any(vm_pu.min()<min_vm_pu):
         print("voltage below minimum")
            
    elif c==uplimit5:
        print("no maximum has been found")
        
c19=c
EVs=c-c19        
print(EVs)        
        
#creating our net
net = nw.create_cigre_network_mv()


#Creating buses and trafos

pp.create_bus(net, name='Bus CS 14', vn_kv=0.4, type='b', geodata=((10,3.5)))
pp.create_bus(net, name='Bus CS 11', vn_kv=0.4, type='b', geodata=((2.5,7)))
pp.create_bus(net, name='Bus CS 5', vn_kv=0.4, type='b', geodata=((-0.5,7)))
pp.create_bus(net, name='Bus CS 4', vn_kv=0.4, type='b', geodata=((1,9)))
pp.create_bus(net, name='Bus CS 9', vn_kv=0.4, type='b', geodata=((6,3.5)))


pp.create_transformer(net, hv_bus=14, lv_bus= pp.get_element_index(net,'bus', 'Bus CS 14'), name='CS 14 - 0.63MVA', std_type= '0.63 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=11, lv_bus=pp.get_element_index(net, 'bus', 'Bus CS 11'),  name='CS 11 - 0.4MVA',std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=5, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 5'),   name='CS 5 - 0.4MVA', std_type= '0.4 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=4, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 4'),   name='CS 4 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')
pp.create_transformer(net, hv_bus=9, lv_bus=pp.get_element_index (net, 'bus', 'Bus CS 9'),   name='CS 9 - 0.25MVA', std_type= '0.25 MVA 20/0.4 kV')


for i in range (1,c15-1):
    l = str(i)
    pp.create_load(net, bus=15, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c15,c16):
    l = str(i)
    pp.create_load(net, bus=16, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c16,c17):
   l = str(i)
   pp.create_load(net, bus=17, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)

for i in range (c17,c18):
    l = str(i)
    pp.create_load(net, bus=18, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
for i in range (c18,c19):
    l = str(i)
    pp.create_load(net, bus=19, p_mw=0.0066, q_mvar=0, name="Vehicle "+l)
    
output_dir = os.path.join("time_series_ev")
print("Results can be found in your local temp folder: {}".format(output_dir))
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
timeseries_example(output_dir)

ll_file = os.path.join(output_dir, "res_line", "loading_percent.xlsx")
line_loading = pd.read_excel(ll_file, index_col=0)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.xlsx")
vm_pu = pd.read_excel(vm_pu_file, index_col=0)

0
Results can be found in your local temp folder: time_series_ev



100%|███████████████████████████████████████████| 24/24 [00:00<00:00, 77.26it/s]


In [147]:
vm_pu.min()

0     1.030000
1     1.019801
2     1.006982
3     0.986634
4     0.986120
5     0.985761
6     0.985401
7     0.982000
8     0.982040
9     0.980994
10    0.978801
11    0.977949
12    1.018629
13    0.997813
14    0.985152
15    0.900496
16    0.899448
17    0.983297
18    0.985710
19    0.978998
dtype: float64

In [60]:
all(vm_pu.min()<min_vm_pu)

False

In [607]:
all(vm_pu.max())<max_vm_pu and all(line_loading.max())<100 and all(vm_pu.min())>min_vm_pu and c<450

False